In [ ]:
import numpy as np
import pandas as pd

# for jupyter notebook widgets
import ipywidgets as widgets
from ipywidgets import interact
from ipywidgets import interact_manual

In [ ]:
#pip install scikit-surprise
from surprise import Dataset, Reader

In [ ]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [ ]:
movies.shape, ratings.shape

In [ ]:
movies.head(5)

In [ ]:
ata = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader=Reader(rating_scale = (1,5)))

In [ ]:

# Lets Build the Training Dataset
train = data.build_full_trainset()

In [ ]:
from surprise import KNNWithMeans
# Item-Item collaborative
my_sim_option = {'name':'pearson', 'user_based':True}
# KNN model as backend 
algo = KNNWithMeans(k = 15, min_k = 5, sim_options = my_sim_option, verbose = True)

In [ ]:
# lets check the Accuracy of the Model
from surprise.model_selection import  cross_validate

# Cross validation 
results = cross_validate(algo = algo, 
                         data = data, 
                         measures=['RMSE'], 
                         cv = 5, 
                         return_train_measures=True)
    
print(results['test_rmse'].mean())

In [ ]:
algo.fit(train)

In [ ]:
# Create a Dictionary to Map the Movie Id and Movie Names
movie_id_to_title_map = {}
for m_id , title in zip(movies['movieId'].values , movies['title'].values):
    movie_id_to_title_map[str(m_id)] = title
{k: movie_id_to_title_map[f'{k}'] for k in range(1, 10)}

In [ ]:
# how much the user id - 1 would rate item id 31 ?
val = algo.predict(uid = '1', iid = '31')
print(movie_id_to_title_map[val[1]] , val[3])

In [ ]:

from collections import defaultdict
def get_top_n(predictions, n = 10):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

In [ ]:
# Build an Iterable Testset, Direct predictions on Train would throw errors 
testdata = train.build_anti_testset()
predictions = algo.test(testdata)
top_n = get_top_n(predictions, n = 10)

In [ ]:
predictions[0]

In [ ]:
# top 10 recommendation for user 1
{(movie_id_to_title_map[x], r ) for (x, r) in top_n['2']}